<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# DKN : Deep Knowledge-Aware Network for News Recommendation

DKN \[1\] is a deep learning model which incorporates information from knowledge graph for better news recommendation. Specifically, DKN uses TransX \[2\] method for knowledge graph representation learning, then applies a CNN framework, named KCNN, to combine entity embedding with word embedding and generate a final embedding vector for a news article. CTR prediction is made via an attention-based neural scorer. 

## Properties of DKN:

- DKN is a content-based deep model for CTR prediction rather than traditional ID-based collaborative filtering. 
- It makes use of knowledge entities and common sense in news content via joint learning from semantic-level and knowledge-level representations of news articles.
- DKN uses an attention module to dynamically calculate a user's aggregated historical representaition.


## Data format

DKN takes several files as input as follows:

- **training / validation / test files**: each line in these files represents one instance. Impressionid is used to evaluate performance within an impression session, so it is only used when evaluating, you can set it to 0 for training data. The format is : <br> 
`[label] [userid] [CandidateNews]%[impressionid] `<br> 
e.g., `1 train_U1 N1%0` <br> 

- **user history file**: each line in this file represents a users' click history. You need to set `history_size` parameter in the config file, which is the max number of user's click history we use. We will automatically keep the last `history_size` number of user click history, if user's click history is more than `history_size`, and we will automatically pad with 0 if user's click history is less than `history_size`. the format is : <br> 
`[Userid] [newsid1,newsid2...]`<br>
e.g., `train_U1 N1,N2` <br> 

- **document feature file**: It contains the word and entity features for news articles. News articles are represented by aligned title words and title entities. To take a quick example, a news title may be: <i>"Trump to deliver State of the Union address next week"</i>, then the title words value may be `CandidateNews:34,45,334,23,12,987,3456,111,456,432` and the title entitie value may be: `entity:45,0,0,0,0,0,0,0,0,0`. Only the first value of entity vector is non-zero due to the word "Trump". The title value and entity value is hashed from 1 to `n` (where `n` is the number of distinct words or entities). Each feature length should be fixed at k (`doc_size` parameter), if the number of words in document is more than k, you should truncate the document to k words, and if the number of words in document is less than k, you should pad 0 to the end. 
the format is like: <br> 
`[Newsid] [w1,w2,w3...wk] [e1,e2,e3...ek]`

- **word embedding/entity embedding/ context embedding files**: These are `*.npy` files of pretrained embeddings. After loading, each file is a `[n+1,k]` two-dimensional matrix, n is the number of words(or entities) of their hash dictionary, k is dimension of the embedding, note that we keep embedding 0 for zero padding. 

In this experiment, we used GloVe\[4\] vectors to initialize the word embedding. We trained entity embedding using TransE\[2\] on knowledge graph and context embedding is the average of the entity's neighbors in the knowledge graph.<br>

## MIND dataset

MIND dataset\[3\] is a large-scale English news dataset. It was collected from anonymized behavior logs of Microsoft News website. MIND contains 1,000,000 users, 161,013 news articles and 15,777,377 impression logs. Every news article contains rich textual content including title, abstract, body, category and entities. Each impression log contains the click events, non-clicked events and historical news click behaviors of this user before this impression.

A smaller version, [MIND-small](https://azure.microsoft.com/en-us/services/open-datasets/catalog/microsoft-news-dataset/), is a small version of the MIND dataset by randomly sampling 50,000 users and their behavior logs from the MIND dataset.

The datasets contains these files for both training and validation data:

#### behaviors.tsv

The behaviors.tsv file contains the impression logs and users' news click hostories. It has 5 columns divided by the tab symbol:

+ Impression ID. The ID of an impression.
+ User ID. The anonymous ID of a user.
+ Time. The impression time with format "MM/DD/YYYY HH:MM:SS AM/PM".
+ History. The news click history (ID list of clicked news) of this user before this impression.
+ Impressions. List of news displayed in this impression and user's click behaviors on them (1 for click and 0 for non-click).

One simple example: 

`1    U82271    11/11/2019 3:28:58 PM    N3130 N11621 N12917 N4574 N12140 N9748    N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25810-0 N20820-0 N6885-0 N27294-0 N18835-0 N16945-0 N7410-0 N23967-0 N22679-0 N20532-0 N26651-0 N22078-0 N4098-0 N16473-0 N13841-0 N15660-0 N25787-0 N2315-0 N1615-0 N9087-0 N23880-0 N3600-0 N24479-0 N22882-0 N26308-0 N13594-0 N2220-0 N28356-0 N17083-0 N21415-0 N18671-0 N9440-0 N17759-0 N10861-0 N21830-0 N8064-0 N5675-0 N15037-0 N26154-0 N15368-1 N481-0 N3256-0 N20663-0 N23940-0 N7654-0 N10729-0 N7090-0 N23596-0 N15901-0 N16348-0 N13645-0 N8124-0 N20094-0 N27774-0 N23011-0 N14832-0 N15971-0 N27729-0 N2167-0 N11186-0 N18390-0 N21328-0 N10992-0 N20122-0 N1958-0 N2004-0 N26156-0 N17632-0 N26146-0 N17322-0 N18403-0 N17397-0 N18215-0 N14475-0 N9781-0 N17958-0 N3370-0 N1127-0 N15525-0 N12657-0 N10537-0 N18224-0 `

#### news.tsv

The news.tsv file contains the detailed information of news articles involved in the behaviors.tsv file. It has 7 columns, which are divided by the tab symbol:

+ News ID
+ Category
+ SubCategory
+ Title
+ Abstract
+ URL
+ Title Entities (entities contained in the title of this news)
+ Abstract Entities (entites contained in the abstract of this news)

One simple example: 

`N46466    lifestyle    lifestyleroyals    The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By    Shop the notebooks, jackets, and more that the royals can't live without.    https://www.msn.com/en-us/lifestyle/lifestyleroyals/the-brands-queen-elizabeth,-prince-charles,-and-prince-philip-swear-by/ss-AAGH0ET?ocid=chopendata    [{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]    [] `

#### entity_embedding.vec & relation_embedding.vec

The entity_embedding.vec and relation_embedding.vec files contain the 100-dimensional embeddings of the entities and relations learned from the subgraph (from WikiData knowledge graph) by TransE method. In both files, the first column is the ID of entity/relation, and the other columns are the embedding vector values.

One simple example: 

`Q42306013  0.014516 -0.106958 0.024590 ... -0.080382`


## DKN architecture

The following figure shows the architecture of DKN.

![](https://recodatasets.blob.core.windows.net/images/dkn_architecture.png)

DKN takes one piece of candidate news and one piece of a user’s clicked news as input. For each piece of news, a specially designed KCNN is used to process its title and generate an embedding vector. KCNN is an extension of traditional CNN that allows flexibility in incorporating symbolic knowledge from a knowledge graph into sentence representation learning. 

With the KCNN, we obtain a set of embedding vectors for a user’s clicked history. To get final embedding of the user with
respect to the current candidate news, we use an attention-based method to automatically match the candidate news to each piece
of his clicked news, and aggregate the user’s historical interests with different weights. The candidate news embedding and the user embedding are concatenated and fed into a deep neural network (DNN) to calculate the predicted probability that the user will click the candidate news.

## Global settings and imports

In [28]:
import sys
sys.path.append("../../")

import os
from tempfile import TemporaryDirectory
import logging
import papermill as pm
import tensorflow as tf

from reco_utils.dataset.download_utils import maybe_download
from reco_utils.dataset.mind import (download_mind, 
                                     extract_mind, 
                                     read_clickhistory, 
                                     get_train_input, 
                                     get_valid_input, 
                                     get_user_history,
                                     get_words_and_entities,
                                     generate_embeddings) 
from reco_utils.recommender.deeprec.deeprec_utils import prepare_hparams
from reco_utils.recommender.deeprec.models.dkn import DKN
from reco_utils.recommender.deeprec.io.dkn_iterator import DKNTextIterator

print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
Tensorflow version: 1.15.2


In [29]:
# Temp dir

BASE_DIR = os.getcwd()
# Logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter("%(asctime)s %(levelname)s: %(message)s", datefmt='%I:%M:%S')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [30]:
# Mind parameters
MIND_SIZE = "demo"

# DKN parameters
epochs = 3
history_size = 50
batch_size = 100

# Paths

data_path = BASE_DIR+'\data'
train_file = os.path.join(data_path, "train_mind_demo.txt")
valid_file = os.path.join(data_path, "valid_mind_demo.txt")
user_history_file = os.path.join(data_path, "user_history.txt")
infer_embedding_file = os.path.join(data_path, "infer_embedding.txt")


## Data preparation

In this example, let's go through a real case on how to apply DKN on a raw news dataset from the very beginning. We will download a copy of open-source MIND dataset, in its original raw format. Then we will process the raw data files into DKN's input data format, which is stated previously. 

In [31]:
#train_zip, valid_zip = download_mind(size=MIND_SIZE, dest_path=data_path)

train_path = data_path+'\\train'

valid_path = data_path+'\\valid'


In [32]:
print(train_path)

C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\train


In [6]:
train_session, train_history = read_clickhistory(train_path, "behaviors.tsv")
valid_session, valid_history = read_clickhistory(valid_path, "behaviors.tsv")
get_train_input(train_session, train_file)
get_valid_input(valid_session, valid_file)
get_user_history(train_history, valid_history, user_history_file)

05:19:12 INFO: Train file C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\train_mind.txt successfully generated
05:19:13 INFO: Validation file C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\valid_mind.txt successfully generated
05:19:13 INFO: User history file C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\user_history.txt successfully generated


In [33]:
train_news = os.path.join(train_path, "news.tsv")
valid_news = os.path.join(valid_path, "news.tsv")
news_words, news_entities = get_words_and_entities(train_news, valid_news)

In [34]:
train_entities = os.path.join(train_path, "entity_embedding.vec")
valid_entities = os.path.join(valid_path, "entity_embedding.vec")
news_feature_file, word_embeddings_file, entity_embeddings_file = generate_embeddings(
    data_path,
    news_words,
    news_entities,
    train_entities,
    valid_entities,
    max_sentence=10,
    word_embedding_dim=100,
)

07:23:32 INFO: Downloading glove...
07:23:32 INFO: File C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\glove.6B.zip already downloaded
07:23:32 INFO: Loading glove with embedding dimension 100...
07:23:45 INFO: Reading train entities...
07:23:46 INFO: Reading valid entities...
07:23:46 INFO: Generating word and entity indexes...
07:23:46 INFO: Generating word embeddings...
07:23:46 INFO: Generating entity embeddings...
07:23:46 INFO: Saving word and entity features in C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\doc_feature.txt
07:23:47 INFO: Saving word embeddings in C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\word_embeddings_5w_100.npy
07:23:47 INFO: Saving word embeddings in C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\entity_embeddings_5w_100.npy


In [35]:
print(news_feature_file)

C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\doc_feature.txt


## Create hyper-parameters

In [37]:
yaml_file = maybe_download(url="https://recodatasets.blob.core.windows.net/deeprec/deeprec/dkn/dkn_MINDsmall.yaml", 
                           work_directory=data_path)
hparams = prepare_hparams(yaml_file,
                          news_feature_file=news_feature_file,
                          user_history_file=user_history_file,
                          wordEmb_file=word_embeddings_file,
                          entityEmb_file=entity_embeddings_file,
                          epochs=epochs,
                          history_size=history_size,
                          batch_size=batch_size)

07:24:00 INFO: File C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\dkn_MINDsmall.yaml already downloaded


## Train the DKN model

In [38]:
model = DKN(hparams, DKNTextIterator)

In [39]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:0.48983963451190704
eval info: auc:0.6316, group_auc:0.5952, mean_mrr:0.2486, ndcg@10:0.3365, ndcg@5:0.2675
at epoch 1 , train time: 787.3 eval time: 352.3
at epoch 2
train info: logloss loss:0.4680527504383489
eval info: auc:0.6385, group_auc:0.6048, mean_mrr:0.2579, ndcg@10:0.3489, ndcg@5:0.2789
at epoch 2 , train time: 810.2 eval time: 351.9
at epoch 3
train info: logloss loss:0.45819693846655934
eval info: auc:0.6488, group_auc:0.616, mean_mrr:0.2619, ndcg@10:0.3547, ndcg@5:0.2876
at epoch 3 , train time: 723.6 eval time: 352.1


## Evaluate the DKN model

In [58]:
test_file = os.path.join(data_path, "test_file3.txt")
output = os.path.join(data_path, "output4.txt")

In [59]:
model.predict(test_file,output)

In [110]:
import pandas as pd
news = pd.read_csv(train_news, sep='\t', names=['News ID', 'Category', 'SubCategory', 'Title', 'Abstract', 'URL', 'Title Entities', 'Abstract Entities'])
news = news.dropna(axis=0, how='any')

result_path=data_path+'\\result2.CSV'
print(result_path)
result=pd.read_csv(result_path,names=['Impression', 'UserID', 'NewsID', 'Score'])
NewsID=result['NewsID'].values[0:10]


ID=news['News ID'].values
df=news.copy()
ilist=[]
for i in range(len(ID)):
    id = ID[i]
    if id in NewsID:
        ilist.append(i)

df.iloc[ilist]

C:\Users\雷雨寒\recommenders-master\examples\02_model_content_based_filtering\data1\result2.CSV
['N22990' 'N18892' 'N18011' 'N14982' 'N13428' 'N18291' 'N15276' 'N26059'
 'N14475' 'N12985']
7036
17561
18031
19948
20460
22538
24945
25150


,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities
7324,N26059,sports,football_nfl,Odell Beckham's Comments After Browns-Patriots...,When the New York Giants shockingly decided to...,https://www.msn.com/en-us/sports/football_nfl/...,"[{""Label"": ""Odell Beckham Jr."", ""Type"": ""P"", ""...","[{""Label"": ""Odell Beckham Jr."", ""Type"": ""P"", ""..."
18355,N14982,sports,football_nfl,"Browns, Steelers brawl at end of Cleveland's 2...",Cleveland defense end Myles Garrett ripped off...,https://www.msn.com/en-us/sports/football_nfl/...,"[{""Label"": ""Cleveland Browns"", ""Type"": ""O"", ""W...","[{""Label"": ""Cleveland Browns"", ""Type"": ""O"", ""W..."
18847,N13428,sports,football_nfl,Packers' Preston Smith buying Rolexes for defe...,Preston Smith's 10th sack of the season could ...,https://www.msn.com/en-us/sports/football_nfl/...,"[{""Label"": ""Preston Smith (linebacker)"", ""Type...","[{""Label"": ""Preston Smith (linebacker)"", ""Type..."
20893,N18291,news,newscrime,"Father, daughter charged with raping woman, le...",A father and daughter were arrested in Los Ang...,https://www.msn.com/en-us/news/newscrime/fathe...,[],"[{""Label"": ""North Las Vegas, Nevada"", ""Type"": ..."
21434,N18892,music,music-celebrity,"Wynonna Judd's Daughter, 23, Released from Pri...",Wynonna Judd's Daughter Released from Prison 6...,https://www.msn.com/en-us/music/music-celebrit...,[],[]
23633,N14475,tv,tvnews,"Rip Taylor's Cause of Death Revealed, Memorial...",The comedian died at the age of 84 last month.,https://www.msn.com/en-us/tv/tvnews/rip-taylor...,"[{""Label"": ""Rip Taylor"", ""Type"": ""P"", ""Wikidat...",[]
26197,N22990,music,musicnews,Broadway Actress Laurel Griggs Dies at Age 13,"Broadway star Laurel Griggs has died, accordin...",https://www.msn.com/en-us/music/musicnews/broa...,"[{""Label"": ""Broadway theatre"", ""Type"": ""F"", ""W...","[{""Label"": ""Broadway theatre"", ""Type"": ""F"", ""W..."
26414,N12985,music,music-celebrity,Broadway Star Laurel Griggs Suffered Asthma At...,"Teen star Laurel Griggs, who passed away on No...",https://www.msn.com/en-us/music/music-celebrit...,"[{""Label"": ""Broadway theatre"", ""Type"": ""F"", ""W...","[{""Label"": ""Once (musical)"", ""Type"": ""W"", ""Wik..."


In [114]:
train_data=open(train_file).readlines()
NewsID=[]
for i in range(len(train_data)):
    line=train_data[i]
    User=line.split(' ')[1].split('train_')[1]
    if User=='U82271':
        NewsID.append(line.split(' ')[2].split('\n')[0])
print(NewsID)

ID=news['News ID'].values
df=news.copy()
ilist=[]
for i in range(len(ID)):
    id = ID[i]
    if id in NewsID:
        ilist.append(i)

df.iloc[ilist]

['N3112', 'N25192', 'N1172', 'N16959', 'N3369', 'N22990', 'N15947', 'N16945', 'N24955', 'N22817', 'N26154', 'N4894', 'N6911', 'N15037', 'N20678', 'N7250', 'N1894', 'N10550', 'N3628', 'N7065']


,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities
0,N3112,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://www.msn.com/en-us/lifestyle/lifestyler...,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
17,N25192,lifestyle,lifestyleroyals,Every outfit Duchess Kate has worn in 2019,See Kate Middleton's style choices this year f...,https://www.msn.com/en-us/lifestyle/lifestyler...,[],"[{""Label"": ""Catherine, Duchess of Cambridge"", ..."
66,N1172,lifestyle,lifestyleroyals,The surprising age differences between your fa...,Here are the age differences between Meghan Ma...,https://www.msn.com/en-us/lifestyle/lifestyler...,[],"[{""Label"": ""Meghan, Duchess of Sussex"", ""Type""..."
197,N16959,lifestyle,lifestyleroyals,Queen Elizabeth's Favorite Beauty Products Hav...,"Here, all the brands the British monarch swear...",https://www.msn.com/en-us/lifestyle/lifestyler...,[],[]
5640,N24955,entertainment,humor,The highlights and lowlights from the world of...,"Along with the great films, sitcoms and talk s...",https://www.msn.com/en-us/entertainment/humor/...,[],[]
16958,N1894,health,nutrition,Osteoarthritis Diet: Foods to Eat and Avoid,Some foods can help decrease your inflammation...,https://www.msn.com/en-us/health/nutrition/ost...,[],[]
17972,N26154,tv,tv-celebrity,Katie Couric Says Matt Lauer 'Ultimately Turne...,"Katie Couric discussed Matt Lauer, the Me Too ...",https://www.msn.com/en-us/tv/tv-celebrity/kati...,"[{""Label"": ""Katie Couric"", ""Type"": ""P"", ""Wikid...","[{""Label"": ""Katie Couric"", ""Type"": ""P"", ""Wikid..."
19079,N16945,health,weightloss,Ree Drummond Says a Low-Carb Diet Is the Secre...,The Pioneer Woman's new cookbook is out now!,https://www.msn.com/en-us/health/weightloss/re...,"[{""Label"": ""Ree Drummond"", ""Type"": ""P"", ""Wikid...",[]
20156,N15037,tv,tv-celebrity,Joe Giudice Calls His Family 'Italian Strong' ...,Joe Giudice Shares Sweet Photo During Family R...,https://www.msn.com/en-us/tv/tv-celebrity/joe-...,"[{""Label"": ""Teresa Giudice"", ""Type"": ""P"", ""Wik...","[{""Label"": ""Teresa Giudice"", ""Type"": ""P"", ""Wik..."
20272,N7250,tv,tv-celebrity,Sarah Hyland documents painful tattoo removal,It looks like she's removing a dinosaur tattoo,https://www.msn.com/en-us/tv/tv-celebrity/sara...,"[{""Label"": ""Sarah Hyland"", ""Type"": ""P"", ""Wikid...",[]


In [11]:
res = model.run_eval(valid_file)
print(res)

{'auc': 0.6474, 'group_auc': 0.623, 'mean_mrr': 0.2834, 'ndcg@5': 0.3048, 'ndcg@10': 0.3723}


In [ ]:
pm.record("res", res)

## Document embedding inference API

After training, you can get document embedding through this document embedding inference API. The input file format is same with document feature file. The output file fomrat is: `[Newsid] [embedding]`

In [ ]:
model.run_get_embedding(news_feature_file, infer_embedding_file)

## Results on large MIND dataset

Here are performances using the large MIND dataset (1,000,000 users, 161,013 news articles and 15,777,377 impression logs). 

| Models | g-AUC | MRR |NDCG@5 | NDCG@10 |
| :------| :------: | :------: | :------: | :------ |
| LibFM | 0.5993 | 0.2823 | 0.3005 | 0.3574 |
| Wide&Deep | 0.6216 | 0.2931 | 0.3138 | 0.3712 |
| DKN | 0.6436 | 0.3128 | 0.3371 | 0.3908|


Note that the results of DKN are using Microsoft recommender and the results of the first two models come from the MIND paper \[3\].
We compare the results on the same test dataset. 

One epoch takes 6381.3s (5066.6s for training, 1314.7s for evaluating) for DKN on GPU. Hardware specification for running the large dataset: <br>
GPU: Tesla P100-PCIE-16GB <br>
CPU: 6 cores Intel(R) Xeon(R) CPU E5-2690 v4 @ 2.60GHz

## References

\[1\] Wang, Hongwei, et al. "DKN: Deep Knowledge-Aware Network for News Recommendation." Proceedings of the 2018 World Wide Web Conference on World Wide Web. International World Wide Web Conferences Steering Committee, 2018.<br>
\[2\] Knowledge Graph Embeddings including TransE, TransH, TransR and PTransE. https://github.com/thunlp/KB2E <br>
\[3\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[4\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/